In [ ]:
import csv
import pandas as pd
import datetime as dt
import statistics as stats
import matplotlib.pyplot as plt

import numpy as np

In [ ]:
#préparer le dataframe
df = pd.read_csv('20220419.csv', sep = ';', encoding='latin1')
df = df[:-1] #La dernière ligne est vide.
drop_list = ['Identifiant','Commentaire','Statut','Type de document','Date du fichier','Page blanche',
             'Association','Ordre chronologique']
#may need to drop 'Unnamed: 12 at some point


df.drop(drop_list, axis=1, inplace=True)
df=df.rename(columns = {'Priorité':"PRIORITE",'Provenance':"PROVENANCE",
                        "Date d'injection":'ENVOI',"Date dern. modif." : "FIN", 'Temps écoulé':'DELAI'})

for count,prio in enumerate(df['PRIORITE']):
    if df.loc[count,'PRIORITE'] == 'STAT- HND':
        df.loc[count,'PRIORITE'] = 'Stat'
    elif df.loc[count,'PRIORITE'] == 'RÉG-HND':
        df.loc[count,'PRIORITE'] = 'Regulier'
    elif df.loc[count,'PRIORITE'] == ' ATTENTE-HND':
        df.loc[count,'PRIORITE'] = 'Attente'
    elif df.loc[count,'PRIORITE'] == 'CORREC-HND':
        df.loc[count,'PRIORITE'] = 'Correction'
    elif df.loc[count,'PRIORITE'] == 'URGENCE-HND':
        df.loc[count,'PRIORITE'] = 'Urgence'
    elif df.loc[count,'PRIORITE'] == 'SINT-HND':
        df.loc[count,'PRIORITE'] = 'Soins'
    elif df.loc[count,'PRIORITE'] == 'ATB-HND':
        df.loc[count,'PRIORITE'] = 'Antibio'

#mettre les heures en datetime pour permettre une filtration
for count, time in enumerate(df["ENVOI"]):
    df.loc[count,"ENVOI"] = dt.datetime.strptime(df.loc[count,"ENVOI"],'%Y-%m-%d %H:%M')
#automatiser les plages de dates
datemin = min(df.ENVOI).strftime('%d-%m-%Y')
datemax = max(df.ENVOI).strftime('%d-%m-%Y')


In [ ]:
#configurer le filtre d'heures d'ouverture
ouverture_sem = dt.datetime.time(dt.datetime(1900,1,1,7,30))
fermeture_sem = dt.datetime.time(dt.datetime(1900,1,1,19,0))
ouverture_fds = dt.datetime.time(dt.datetime(1900,1,1,8,0))
fermeture_fds = dt.datetime.time(dt.datetime(1900,1,1,17,0))

def timerange(start,current,end):
    return start <= current <= end

semaine = ['Monday','Tuesday','Wednesday','Thursday','Friday']
fds = ['Saturday','Sunday']
feries = ['2021-06-24','2021-07-02','2021-09-06','2021-10-11','2021-12-23','2021-12-24','2021-12-30','2021-12-31',
          '2022-04-15','2022-04-18','2022-05-23','2022-06-24']

#appliquer un filtre sur le jour de la semaine afin de pouvoir séparer les données en semaine vs fds

for count,item in enumerate(df['ENVOI']):
    if df.loc[count,'ENVOI'].strftime('%A') in semaine:
        if df.loc[count,'ENVOI'].strftime('%Y-%m-%d') in feries: #pour tenir compte des journées fériées comme la fds
            if timerange(ouverture_fds, df.loc[count,'ENVOI'].time(), fermeture_fds) == True:
                df.loc[count,'JOUR'] = "fds"
        elif timerange(ouverture_sem, df.loc[count,'ENVOI'].time(), fermeture_sem) == True:
            df.loc[count,'JOUR'] = "sem"
    if df.loc[count,'ENVOI'].strftime('%A') in fds:
        if timerange(ouverture_fds, df.loc[count,'ENVOI'].time(), fermeture_fds) == True:
            df.loc[count,'JOUR'] = "fds"

#convertir les strings d'heures "Xh YYm" en int

for count,item in enumerate(df['DELAI']):
    heures = int(df.loc[count,'DELAI'].split(' ')[0].split('h')[0])
    minutes = int(df.loc[count,'DELAI'].split(' ')[1].split('m')[0])
    time = heures*60 + minutes
    df.loc[count,'DELAI'] = time

In [ ]:
#créer les df pour l'analyse finale
dic_semaine = {'PRIORITE':[],'PROVENANCE':[],'ENVOI':[],'FIN':[],'DELAI':[]}
dic_fds = {'PRIORITE':[],'PROVENANCE':[],'ENVOI':[],'FIN':[],'DELAI':[]}
dic_total = {'PRIORITE':[],'PROVENANCE':[],'ENVOI':[],'FIN':[],'DELAI':[]}

for count, jour in enumerate(df['JOUR']):
    if jour == 'sem':
        dic_semaine['PRIORITE'].append(df.loc[count,'PRIORITE'])
        dic_semaine['PROVENANCE'].append(df.loc[count,'PROVENANCE'])
        dic_semaine['ENVOI'].append(df.loc[count,'ENVOI'])
        dic_semaine['FIN'].append(df.loc[count,'FIN'])
        dic_semaine['DELAI'].append(df.loc[count,'DELAI'])
    if jour == 'fds':
        dic_fds['PRIORITE'].append(df.loc[count,'PRIORITE'])
        dic_fds['PROVENANCE'].append(df.loc[count,'PROVENANCE'])
        dic_fds['ENVOI'].append(df.loc[count,'ENVOI'])
        dic_fds['FIN'].append(df.loc[count,'FIN'])
        dic_fds['DELAI'].append(df.loc[count,'DELAI'])
    if jour in ['sem','fds']:
        dic_total['PRIORITE'].append(df.loc[count,'PRIORITE'])
        dic_total['PROVENANCE'].append(df.loc[count,'PROVENANCE'])
        dic_total['ENVOI'].append(df.loc[count,'ENVOI'])
        dic_total['FIN'].append(df.loc[count,'FIN'])
        dic_total['DELAI'].append(df.loc[count,'DELAI'])
df_semaine = pd.DataFrame(data = dic_semaine)
df_fds = pd.DataFrame(data = dic_fds)
df_total = pd.DataFrame(data = dic_total)

In [ ]:
#j'additionne le temps total pour chacune des priorités dans dic_delai, en plus de compter le nombre d'apparitions
#dans dic_count afin de pouvoir y accéder rapidement par la suite. Il y a sûrement une meilleure façon de le faire

#on retrouve deux counts ici. le premier dic_count sert à calculer une moyenne vs count qui sert à passer à iterate à
#travers le df.

def comparaison(storage,dic_delai):
    '''
    storage being a list of the info I want to put in the file
    then comparing the present date with the old one
    '''
    dates = []
    with open('oldstats.csv',newline = '', encoding= 'latin1') as file:
        reader = csv.reader(file, delimiter=';')
        for row in reader:
            dates.append(row[0])

    with open('oldstats.csv','a',newline = '', encoding= 'latin1') as file:
        writer = csv.writer(file, delimiter = ';')
        if storage[0] not in dates:
            writer.writerow(storage)

    df = pd.read_csv('oldstats.csv', sep = ';', encoding='latin1')
    dic_comparaison = {}

    dic_comparaison['nb_rx'] = df.loc[len(df)-2,'nb_rx']
    dic_comparaison['periode'] = df.loc[len(df)-2,'periode']

    new_date = datetime.strptime(df.loc[len(df)-1,'date'],'%d-%m-%Y')
    old_date = datetime.strptime(df.loc[len(df)-2,'date'],'%d-%m-%Y')
    periode = new_date-old_date     #periode being a timedeta object
    periode = int(periode.days)


    for key in dic_delai.keys():
        comparaison = round(dic_delai[key] - df.loc[len(df)-2,key],2) #returns previous entry as len(df) includes the indexes

        if comparaison >= 0:
            dic_comparaison[key] = f'+{comparaison} minutes sur {periode} jours'
        if comparaison < 0:
            dic_comparaison[key] = f'{comparaison} minutes sur {periode} jours'

    return dic_comparaison

def delai(df,cutoff):
    dic_delai = {'Global': df[df['DELAI']<=cutoff]['DELAI'].sum()}
    dic_count = {'Global': df[df['DELAI']<=cutoff]['DELAI'].count()}
    dic_median = {'Global': [stats.median(df['DELAI'])]}
    for count,prio in enumerate(df['PRIORITE']):
        if df.loc[count,'PRIORITE'] not in dic_delai:
            if df.loc[count,'DELAI'] <= cutoff:
                dic_delai[prio] = df.loc[count,'DELAI']
                dic_count[prio] = 1
            dic_median[prio] = []
            dic_median[prio].append(df.loc[count,'DELAI'])
        elif df.loc[count,'PRIORITE'] in dic_delai:
            if df.loc[count,'DELAI'] <= cutoff:
                dic_delai[prio] += df.loc[count,'DELAI']
                dic_count[prio] += 1
            dic_median[prio].append(df.loc[count,'DELAI'])

    #setting up the graph df
    hist_plot = {'priorite':[],'Moyenne':[],'Médiane':[]}
    labels = []

    cut4 = df[df['DELAI']<=240]['DELAI'].count()
    proportion_cut4 = str(round(100*cut4/df['DELAI'].count(),2))+'%'
    duration =  (df.ENVOI.max()-df.ENVOI.min()).days+1

    for key in dic_delai:
        dic_delai[key] = round(dic_delai[key]/dic_count[key],2)
        hist_plot['priorite'].append(key)
        hist_plot['Moyenne'].append(dic_delai[key]) #prepping graph
        hist_plot['Médiane'].append(stats.median(sorted(dic_median[key]))) #prepping graph
        labels.append(key)

    store_stat = [datemax,dic_delai['Global'],dic_delai['Regulier'],dic_delai['Antibio'],
                  dic_delai['Stat'], dic_delai['Correction'],dic_delai['Urgence'],dic_delai['Attente'],
                  dic_delai['Soins'],cut4,duration]
    dic_comp = comparaison(store_stat,dic_delai)

    print(f"Délais moyen et médian de complétion des feuilles à la distribution (en minutes) - {df['DELAI'].count()} feuilles.")
    print(f"Le nombre de feuilles dont le délai est inférieur à 4h est de {cut4}, soit {proportion_cut4} (vs {dic_comp['nb_rx']} sur {dic_comp['periode']} jours).\n")

    for key in dic_delai:
        print(f'{key}\nMoyenne: {dic_delai[key]} ({dic_comp[key]}) - Médiane: {stats.median(sorted(dic_median[key]))}')
#         print(f'{key} - delai moyen : {dic_delai[key]}, delai median : {stats.median(sorted(dic_median[key]))}')
#         print(f'min: {min(sorted(dic_median[key]))}  max: {max(sorted(dic_median[key]))}\n')


    #histogramme des priorités
    hist_plot = pd.DataFrame(data = hist_plot).set_index('priorite',drop=False) #creates df for plot
    fig = plt.figure(figsize=(15,10)) #creates matplotlib fig
    ax = fig.add_subplot(111) #creates 1x1 axes
    width = 0.2

    hist_plot['Moyenne'].plot(kind='bar', color='green', ax=ax, width = width, position=1)
    hist_plot['Médiane'].plot(kind='bar', color='blue', ax=ax, width = width, position=0)

    #setting ticks interval
    major_ticks = np.arange(0,max(hist_plot['Médiane'])+10,10)
    minor_ticks = np.arange(0,max(hist_plot['Médiane'])+10,10)
    ax.set_yticks(major_ticks)
    ax.set_yticks(minor_ticks, minor=True)
    #setting tick thickness
    ax.grid(axis = 'y', which='minor', alpha=0.25)
    ax.grid(axis = 'y', which='major', alpha=0.5)
    ax.grid(axis = 'x', alpha = 0.1)

    plt.legend(frameon=False)

    ax.set_ylabel('Délai(minutes)')
    plt.xlabel('Priorité')
    plt.xticks(rotation='horizontal')
    plt.show()

def erreur(df):
    nb_corrections = len(df[df['PRIORITE'] == 'Correction'])
    total_rx = df.index.max()
    return round(nb_corrections/total_rx*100,2)

def graph_progression():
    graph_df = pd.read_csv('oldstats.csv', sep = ';', encoding = 'latin1')
    graph_df = graph_df[['date','Global','Régulier','Stat','Urgence']].set_index('date')
    graph_df[-8:].plot(figsize=(15,10))
    plt.xlabel('Date')
    plt.ylabel('Temps moyen (minutes)')
    plt.title('Progression des temps de traitement dans le temps')
    plt.show();

In [ ]:
print(f"""
Vous trouverez ci-bas les délais de traitement des ordonnances du {datemin} au {datemax}. Les données sont tirées de NumeRx, dans le module 2-GD (gestion des documents).
Les filtres utilisés sont les suivants :
Statut : ' VALIDÉ' (avec un espace)
Type de document : '02-Rx'
Provenance : 'HND-'
Date d'injection : variable selon la semaine

Les données sont alors extraites dans un document .csv et traitées pour obtenir certaines statistiques selon la priorité au moment de la validation.
Le temps moyen est calculé pour les délais égaux ou inférieurs à 4h lors des heures d'ouverture de la pharmacie. Ce délai a été choisi afin de limiter l'impact que certains feuilles peuvent avoir sur les résultats (classés en retard, rappel tardif du professionel concerné, etc.)
La médiane provient de la distribution complète des feuilles envoyées dans les heures d'ouverture de la pharmacie.

Il faut prendre en considération que le nombre de feuilles retournées en correction est probablement sous-estimé par le fait que les pharmaciens auront parfois tendances à eux-même faire les corrections. Ceci diminue aussi les temps de traitement de certaines feuilles.
De plus, les jours fériés sont catégorisés dans la fin de semaine.
""")

In [ ]:
print('Résultats combinés de la semaine et la fin de semaine')
print(f'Le pourcentage des feuilles retournées en correction est de {erreur(df_total)}%.\n')
delai(df_total,240)

print('Résultats de la semaine')
print(f'Le pourcentage des feuilles retournées en correction est de {erreur(df_semaine)}%.\n')
delai(df_semaine,240)

print('Résultats de la fin de semaine')
print(f'Le pourcentage des feuilles retournées en correction est de {erreur(df_fds)}%.\n')
delai(df_fds,240)

# graph_progression()